In [1]:
import os
# os.environ['CUDA_VISIBLE_DEVICES'] = '0, 1'

from dataloader import *
from models import load_model
from metrics import *
import torch
import torchvision.transforms as transforms
import torchvision.models as models
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import pandas as pd
import numpy as np
from tqdm import tqdm
from datetime import datetime
import scipy
import class_names

import json
import urllib
from preprocessing import preprocess_images_any_dataset

/home/davidva/miniconda3/envs/anaconda/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [18]:
def load_imagenet_labels():
    """
    Fetches the ImageNet class labels from a JSON file hosted online.

    Returns:
        list: A list of class labels where the index corresponds to the class ID in the ImageNet dataset.
    """
    classes_text = 'https://raw.githubusercontent.com/anishathalye/imagenet-simple-labels/master/imagenet-simple-labels.json'
    with urllib.request.urlopen(classes_text) as url:
        imagenet_labels = json.loads(url.read().decode())
    return imagenet_labels


# Register hook to capture the model's embeddings
def get_embedding_hook(batch_idx, batch_size, embed_dir):
    """
    Creates and returns a hook function to capture and save the model's embeddings.

    Returns:
        function: A hook function that can be registered to a PyTorch nn.Module.
    """
    def hook(module, input, output):
        output = output.detach().cpu().numpy()
        for i in range(output.shape[0]):
            # Calculate the overall index of the image in the dataset
            image_idx = batch_idx * batch_size + i
            np.save(os.path.join(embed_dir, f"{image_idx}_embeddings.npy"), output[i])
    return hook


def main():
    # Define device, batch size and directory path for ImageNet validation set
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")    
    data_name = 'imagenet-a'
    base_exp_dir = '/home/davidva/experiments_David'
    val_dir = '/home/davidva/datasets/imagenet-a'

    # Define the batch size
    batch_size = 128

    # Load imagenet labels as real classes
    imagenet_labels = load_imagenet_labels()  # 'tench', 'goldfish', 'great white shark', 'tiger shark', 'hammerhead shark'...

    if data_name in ['imagenet', 'imagenet-v2', 'imagenet-sketch']:
        ds_specific_mask = [True] * 1000
        ds_specific_labels = imagenet_labels  # the classes of the specific dataset that we're working with
    elif data_name == 'imagenet-r':
        ds_specific_r_mask = class_names.imagenet_r_mask
        ds_specific_labels = class_names.imagenet_r_labels
    elif data_name == 'imagenet-a':  # note: if the hidden file .ipynb_checkpoints is inside of the folder of the imagenet-a data,
                                     # it should be removed from there in order to create the DataFolder object
        ds_specific_mask = class_names.imagenet_a_mask
        ds_specific_labels = class_names.imagenet_a_labels
        
    model_names = ['resnet50', 'resnet18', 'resnet34', 'resnet101', 'resnet152', 'vgg16', 'vgg19', 'alexnet', 'resnext', 'wide_resnet', 'densenet121', 'googlenet', 'mobilenet_v2']
    # model_names = ['googlenet', 'mobilenet_v2']

    for model_name in model_names:
        # Experiment name
        curr_time = datetime.now().strftime("%d-%m-%Y_%H-%M-%S")
        exp_base_name = 'predict_' + data_name + '_' + model_name
        exp_name = exp_base_name + '_' + curr_time
        exp_dir = os.path.join(base_exp_dir, exp_name)
        results_dir = os.path.join(exp_dir, 'results')
        embed_dir = os.path.join(results_dir, 'val_embeddings')
    
        # Create experiment directory
        os.makedirs(exp_dir, exist_ok=True)
        os.makedirs(results_dir, exist_ok=True)
        os.makedirs(embed_dir, exist_ok=True)
            
        # Load the ImageNet validation dataset with defined transformations
        val_dataset = ImageFolder(val_dir, transform=preprocess_images_any_dataset(model_name))
        val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=5)

        # Load model
        model = load_model(model_name).to(device)
        
        # Register the hook at the penultimate layer of the model
        if model_name in ['densenet121', 'mobilenet_v2']:
            layer = model.features  # TODO is it correct?
        else:
            layer = model.avgpool
        
        # Prepare DataFrame to store results
        results = []
        total_correct = 0
        batch_num = 0
        all_entropies = np.empty([0])
        
        # Disable gradients for faster inference
        with torch.no_grad():
            for batch_idx, (images, labels) in enumerate(tqdm(val_loader, desc=f"Processing {exp_name}")):
                images = images.to(device)
                labels = labels.to(device)

                handle = layer.register_forward_hook(get_embedding_hook(batch_idx, batch_size, embed_dir))

                # Run forward pass through the model and calculate probabilities
                output = model(images)
                probabilities = torch.nn.functional.softmax(output, dim=1)
                probabilities = probabilities * torch.tensor(ds_specific_mask).to(device)  # using only the classes that are in both datasets
    
                # Get top-5 predictions and confidences
                top5_prob, top5_classes = torch.topk(probabilities, 5)
                entropies = scipy.stats.entropy(probabilities.detach().cpu().numpy(), axis=1)
                all_entropies = np.concatenate((all_entropies, entropies))  # might be really wasteful in terms of run time

                # # Iterate over each image in the batch
                # for i in range(len(images)):
                #     image_path = val_dataset.imgs[i][0]
                #     true_class = val_dataset.classes[labels[i].item()]
                #     predictions = {"image_path": image_path, "true_class": true_class}
                #     for j in range(5):
                #         predictions[f"top{j + 1}_prediction_class"] = val_dataset.classes[top5_classes[i, j].item()]
                #         predictions[f"top{j + 1}_confidence"] = top5_prob[i, j].item()
                #     results.append(predictions)
    
                # Iterate over each image in the batch
                for i in range(len(images)):
                    image_idx = batch_idx * batch_size + i
                    image_path = val_dataset.imgs[i + batch_num * batch_size][0]
                    # true_class = val_dataset.classes[labels[i].item()]
                    true_class = ds_specific_labels[labels[i].item()]  # e.g. labels[i].item()=0. true_class=goldfish
                    predictions = {"image_path": image_path,
                                   "true_class": true_class,
                                   "embeddings_path":  os.path.join(embed_dir, f"{image_idx}_embeddings.npy")}
                    
                    for j in range(5):
                        # predictions[f"top{j+1}_prediction_class"] = val_dataset.classes[top5_classes[i, j].item()]
                        prediction_class = imagenet_labels[top5_classes[i, j].item()]
                        predictions[f"top{j+1}_prediction_class"] = prediction_class
                        predictions[f"top{j+1}_confidence"] = top5_prob[i, j].item()
    
                        # Check if top-1 prediction is correct
                        if j == 0 and prediction_class == true_class:
                            total_correct += 1
                    results.append(predictions)
                batch_num += 1
                handle.remove()  # Unregister the hook after processing the batch
        
        # Convert results to DataFrame and save to CSV
        results_df = pd.DataFrame(results)
        results_df.to_csv(os.path.join(results_dir, f"full_val_predictions_{exp_base_name}.csv"), index=False)
    
        # Calculate average and std of the top1 confidences across the val set2
        avg_confidence = results_df["top1_confidence"].mean()
        std_confidence = results_df["top1_confidence"].std()
        accuracy = total_correct / len(val_dataset)
        average_entropy = np.mean(all_entropies)
        
        # Save average and std confidences in a new DataFrame
        confidence_df = pd.DataFrame({
            "Accuracy": [accuracy],
            "Average Confidence": [avg_confidence],
            "Standard Deviation Confidence": [std_confidence],
            "Average Entropy": [average_entropy]
        })
        confidence_df.to_csv(os.path.join(results_dir, f"val_confidence_summary_{exp_base_name}.csv"), index=False)

        # Handle embeddings
        embeddings = np.array([np.load(file) for file in results_df['embeddings_path']])
        # assuming embeddings is a 4D numpy array
        embeddings = embeddings.reshape(embeddings.shape[0], -1)
        average, covariance = calculate_activation_statistics(embeddings)
        np.save(os.path.join(results_dir, 'embeddings_covariance.npy'), covariance)
        np.save(os.path.join(results_dir, 'embeddings_average.npy'), average)
        # np.save(os.path.join(results_dir, 'tot_embeddings_2D_np_array.npy'), embeddings)
        
        # Create a new DataFrame for embeddings
        # df_embeddings = pd.DataFrame(*******, columns=np.arange(0, embeddings.shape[1]))
        # df_embeddings['true_class'] = results_df['true_class']
        # df_embeddings['predicted_class'] = results_df['top1_prediction_class']
        # df_embeddings['image_path'] = results_df['image_path']
        # # Save as CSV
        # df_embeddings.to_csv(os.path.join(results_dir, f"val_embeddings_2d_{exp_base_name}.csv"), index=False)
        break

In [19]:
if __name__ == '__main__':
    main()

Processing predict_imagenet-a_resnet50_18-07-2023_15-27-30: 100%|█████████████████| 59/59 [01:37<00:00,  1.65s/it]


In [33]:
# for layer in k.children():
#     print(layer)

model_names = ['resnet50', 'resnet18', 'resnet34', 'resnet101', 'resnet152', 'vgg16', 'vgg19', 'alexnet', 'resnext', 'wide_resnet', 'densenet121', 'googlenet', 'mobilenet_v2']

for model_name in model_names:
    k = load_model(model_name)
    layers_list = []
    for name, _ in k.named_children():
        if not name.startswith('params'):
            layers_list.append(name)
    
    print(model_name, layers_list[-2], layers_list[-1])


resnet50 avgpool fc
resnet18 avgpool fc
resnet34 avgpool fc
resnet101 avgpool fc
resnet152 avgpool fc
vgg16 avgpool classifier
vgg19 avgpool classifier
alexnet avgpool classifier
resnext avgpool fc
wide_resnet avgpool fc
densenet121 features classifier
googlenet dropout fc
mobilenet_v2 features classifier


In [23]:
print(load_model('mobilenet_v2'))

MobileNetV2(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(96, eps=